# PIPELINE OF FEATURE EXTRACTION

In [8]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
from skimage import exposure
from scipy import ndimage
from cellpose import models
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage import filters
from skimage import morphology
from skimage import measure
from matplotlib import colors
from importlib import reload
import utility
reload(utility)
from utility import get_hard_disk_path, compute_big_cell_labels, compute_small_cell_labels, calculate_closest_boundary_distances

initial_path = r"E:\data_for_seg\\"
all_labels = ["keep2", "reseed1", "split", "empty", "keep0", "keep1", "reseed0"]
labels_big_cell = ["dead", "keep2", "reseed1", "split"]
labels_small_cell = ["empty", "keep0", "keep1", "reseed0"]
properties_list = ['area', 'mean_intensity', 'solidity', 'convex_area', 'equivalent_diameter', 'perimeter', 'extent', 'max_intensity', 'min_intensity', 'eccentricity']
output_csv = initial_path + "statistics_features_IP.csv"  # Adjust the output file name

def compute_feature_statistics(data, output_csv, properties_list):
    statistics = {}

    other_features = {
        "image_name": data["image_name"].iloc[0],  # Assuming 'image_name' is the same for all rows in data
        "label": data["label"].iloc[0],  # Assuming 'label' is the same for all rows in data
        "count": data["sample"].max(),
        "occupancy": data["occupancy"],
        #"density": data["density"]
    }
    statistics.update(other_features)

    for column_name in properties_list:
        if column_name in data.columns:
            column = data[column_name]

            # Calculate statistics for the current column
            mean_value = column.mean()
            sd_value = column.std()
            q1_value = column.quantile(0.25)  # 25% Quartile
            median_value = column.quantile(0.50)  # 50% Quartile or Median
            q3_value = column.quantile(0.75)  # 75% Quartile

            # Add the statistics to the row dictionary
            statistics.update({
                f"{column_name}_Mean": mean_value,
                f"{column_name}_StdDev": sd_value,
                f"{column_name}_Q1": q1_value,
                f"{column_name}_Median": median_value,
                f"{column_name}_Q3": q3_value,
            })

    # Create a DataFrame from the dictionary
    statistics_df = pd.DataFrame(statistics, index=[0])

    # Save the DataFrame to the output CSV file
    statistics_df.to_csv(output_csv, mode='a', header=not os.path.isfile(output_csv), index=False)

def compute_occupancy(labels, image):
    """
    Compute the occupancy of cells in the image.
    
    Parameters:
    - labels: labeled image
    - image: original grayscale image
    
    Returns:
    - occupancy: ratio of cell area to total image area
    """
    cell_area = np.sum(labels > 0)  # Total number of pixels occupied by cells
    image_area = image.size         # Total number of pixels in the image
    return cell_area / image_area

if os.path.exists(output_csv):
    os.remove(output_csv)

if not os.path.exists(initial_path + "labeled/"):
    os.makedirs(initial_path + "labeled/")

for label in all_labels:
    label_path = os.path.join(initial_path + "labeled", label)
    if not os.path.exists(label_path):
        os.mkdir(label_path)
    if not os.path.exists(os.path.join(label_path, "IP_pipeline")):
        os.mkdir(os.path.join(label_path, "IP_pipeline"))
    os.chdir(os.path.join(initial_path + "z_projection", label))
    files = [file for file in os.listdir() if "Simple" not in file and "filled" not in file]
    print(files)
    for file in files:
        if "Simple" or "filled" not in file:
            image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
            if label in labels_big_cell:
                print(file)
                labels = compute_big_cell_labels(image)
            elif label in labels_small_cell:
                labels = compute_small_cell_labels(image)
            # Generate a colormap for labels
            label_cmap = colors.ListedColormap(np.random.rand(labels.max() + 1, 3))

            # Create a colored label image
            colored_label_image = label_cmap(labels)

            # Convert to uint8 format (0-255)
            colored_label_image = (colored_label_image * 255).astype(np.uint8)

            output_path = os.path.join(label_path, "IP_pipeline", file)
            if not os.path.exists(output_path):
                cv2.imwrite(output_path, cv2.cvtColor(colored_label_image, cv2.COLOR_RGBA2BGRA))

            info_table = pd.DataFrame(
                measure.regionprops_table(
                    labels,
                    intensity_image=image,
                    properties=['label', 'area', 'mean_intensity', 'solidity', 'convex_area', 'equivalent_diameter', 'perimeter', 'extent', 'max_intensity', 'min_intensity', 'eccentricity'],
                )
            )

            occupancy = compute_occupancy(labels, image)
            #density = calculate_closest_boundary_distances(labels)
            #info_table['density'] = density
            info_table['occupancy'] = occupancy  # Add the occupancy to the statistics
            info_table = info_table.rename(columns={"label": "sample"})
            info_table['label'] = label
            info_table['image_name'] = file

            compute_feature_statistics(info_table, output_csv, properties_list)

['s05_no_stack.jpg', 's06_no_stack.jpg', 's07_no_stack.jpg', 's08_no_stack.jpg', 's05.jpg', 's06.jpg', 's07.jpg', 's08.jpg', 's03_no_stack.jpg', 's04_no_stack.jpg', 's12.jpg', 's13.jpg', 's14.jpg', 's12_no_stack.jpg', 's13_no_stack.jpg', 's14_no_stack.jpg', 's09_no_stack.jpg', 's09.jpg', 's19.jpg']
s05_no_stack.jpg
s06_no_stack.jpg
s07_no_stack.jpg
s08_no_stack.jpg
s05.jpg
s06.jpg
s07.jpg
s08.jpg
s03_no_stack.jpg
s04_no_stack.jpg
s12.jpg
s13.jpg
s14.jpg
s12_no_stack.jpg
s13_no_stack.jpg
s14_no_stack.jpg
s09_no_stack.jpg
s09.jpg
s19.jpg
['s13.jpg', 's14.jpg', 's15.jpg', 's17.jpg', 's19.jpg', 's20.jpg', 's21.jpg', 's22.jpg', 's23.jpg', 's24.jpg']
s13.jpg
s14.jpg
s15.jpg
s17.jpg
s19.jpg
s20.jpg
s21.jpg
s22.jpg
s23.jpg
s24.jpg
['s03.jpg', 's04.jpg', 's05.jpg', 's08.jpg', 's07.jpg', 's06.jpg', 's17.jpg', 's16.jpg', 's12.jpg', 's13.jpg', 's18.jpg', 's19.jpg', 's02_no_stack.jpg', 's06_no_stack.jpg', 's09.jpg', 's03_no_stack.jpg', 's04 (2).jpg', 's05 (2).jpg', 's06 (2).jpg', 's08 (2).jpg', 's1

In [11]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
from scipy.spatial.distance import cdist
from skimage import exposure
from scipy import ndimage
from cellpose import models
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage import filters
from skimage import morphology
from skimage import measure
from matplotlib import colors
from importlib import reload
import utility
reload(utility)
from utility import get_hard_disk_path, compute_big_cell_labels, compute_small_cell_labels, calculate_closest_boundary_distances

initial_path = r"E:\data_for_seg\\"
all_labels = ["keep2", "reseed1", "split", "empty", "keep0", "keep1", "reseed0"]
labels_big_cell = ["dead", "keep2", "reseed1", "split"]
labels_small_cell = ["empty", "keep0", "keep1", "reseed0"]
properties_list = ['area', 'mean_intensity', 'solidity', 'convex_area', 'equivalent_diameter', 'perimeter', 'extent', 'max_intensity', 'min_intensity', 'eccentricity']
output_csv = initial_path + "statistics_features_ilastik.csv"  # Adjust the output file name

# --------------------
# ALL FUNCTIONS
# --------------------

def compute_feature_statistics(data, output_csv, properties_list):
    statistics = {}

    other_features = {
        "image_name": data["image_name"].iloc[0],  # Assuming 'image_name' is the same for all rows in data
        "label": data["label"].iloc[0],  # Assuming 'label' is the same for all rows in data
        "count": data["sample"].max(),
        "occupancy": data["occupancy"],
        #"density": data["density"]
    }
    statistics.update(other_features)

    for column_name in properties_list:
        if column_name in data.columns:
            column = data[column_name]

            # Calculate statistics for the current column
            mean_value = column.mean()
            sd_value = column.std()
            min_value = column.min()
            max_value = column.max()

            # Add the statistics to the row dictionary
            statistics.update({
                f"{column_name}_Mean": mean_value,
                f"{column_name}_StdDev": sd_value,
                f"{column_name}_Min": min_value,
                f"{column_name}_Max": max_value,
            })

    # Create a DataFrame from the dictionary
    statistics_df = pd.DataFrame(statistics, index=[0])

    # Save the DataFrame to the output CSV file
    statistics_df.to_csv(output_csv, mode='a', header=not os.path.isfile(output_csv), index=False)

def compute_occupancy(labels, image):
    """
    Compute the occupancy of cells in the image.
    
    Parameters:
    - labels: labeled image
    - image: original grayscale image
    
    Returns:
    - occupancy: ratio of cell area to total image area
    """
    cell_area = np.sum(labels > 0)  # Total number of pixels occupied by cells
    image_area = image.size         # Total number of pixels in the image
    return cell_area / image_area

def extract_features(labels, output_path, image_path):

    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    print(image_path)
    # Generate a colormap for labels
    label_cmap = colors.ListedColormap(np.random.rand(labels.max() + 1, 3))

    # Create a colored label image
    colored_label_image = label_cmap(labels)

    # Convert to uint8 format (0-255)
    colored_label_image = (colored_label_image * 255).astype(np.uint8)

    if not os.path.exists(output_path):
        cv2.imwrite(output_path, cv2.cvtColor(colored_label_image, cv2.COLOR_RGBA2BGRA))

    info_table = pd.DataFrame(
        measure.regionprops_table(
            labels,
            intensity_image=image,
            properties=['label', 'area', 'mean_intensity', 'solidity', 'convex_area', 'equivalent_diameter', 'perimeter', 'extent', 'max_intensity', 'min_intensity', 'eccentricity'],
        )
    )

    occupancy = compute_occupancy(labels, image)
    #density = calculate_closest_boundary_distances(labels)
    #info_table['density'] = density
    info_table['occupancy'] = occupancy  # Add the occupancy to the statistics
    info_table = info_table.rename(columns={"label": "sample"})
    info_table['label'] = label
    info_table['image_name'] = os.path.basename(image_path)

    compute_feature_statistics(info_table, output_csv, properties_list)

# --------------------
# MAIN
# --------------------

if os.path.exists(output_csv):
    os.remove(output_csv)

if not os.path.exists(initial_path + "labeled/"):
    os.makedirs(initial_path + "labeled/")
    

for label in all_labels:
    label_path = os.path.join(initial_path + "labeled", label)
    if not os.path.exists(label_path):
        os.mkdir(label_path)
    os.chdir(os.path.join(initial_path + "z_projection", label))
    if not os.path.exists(os.path.join(label_path, "Ilastik_pipeline")):
        os.mkdir(os.path.join(label_path, "Ilastik_pipeline"))
    files = os.listdir()
    if not os.path.exists("filled_cleaned"):
        for file in files:
            if "Simple" not in file:
                image_path = file
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                labels = compute_small_cell_labels(image)
                output_path = os.path.join(label_path, "Ilastik_pipeline", file)
                extract_features(labels, output_path, image_path)
    elif os.path.exists(os.path.join(initial_path, "z_projection", label, "filled_cleaned")):
        # Define the directory for filled_cleaned files and original_files
        filled_cleaned_dir = os.path.join(initial_path, "z_projection", label, "filled_cleaned")
        original_files_dir = os.path.join(filled_cleaned_dir, "original_files")

        # Get a list of files in filled_cleaned directory
        filled_cleaned_files = os.listdir(filled_cleaned_dir)

        for file in filled_cleaned_files:
            if "labeled_filled" in file:

                # Read the labels file using the full path
                labels_path = os.path.join(filled_cleaned_dir, file)
                labels = cv2.imread(labels_path, cv2.IMREAD_UNCHANGED)

                # Check if the labels were read successfully
                if labels is None:
                    print(f"Failed to read labels from {labels_path}")
                    continue

                labels = labels.astype(int)

                # Create the output path for labeled_cleaned files
                output_path = os.path.join(label_path, "Ilastik_pipeline", file)

                image_path = os.path.join(original_files_dir, file[15:-24] + ".jpg")

                # Check if the image was read successfully
                if image is None:
                    print(f"Failed to read image from {image_path}")
                    continue

                # Now you can extract features with the correct image and labels
                extract_features(labels, output_path, image_path)


Successfully loaded data from E:/data_for_seg/
E:/data_for_seg/z_projection\dead\filled_cleaned\original_files\s09.jpg
E:/data_for_seg/z_projection\dead\filled_cleaned\original_files\s11_no_stack.jpg
E:/data_for_seg/z_projection\dead\filled_cleaned\original_files\s12_no_stack.jpg
E:/data_for_seg/z_projection\dead\filled_cleaned\original_files\s12.jpg
E:/data_for_seg/z_projection\dead\filled_cleaned\original_files\s13_no_stack.jpg
E:/data_for_seg/z_projection\dead\filled_cleaned\original_files\s13.jpg
E:/data_for_seg/z_projection\dead\filled_cleaned\original_files\s01.jpg
E:/data_for_seg/z_projection\dead\filled_cleaned\original_files\s02_no_stack.jpg
E:/data_for_seg/z_projection\dead\filled_cleaned\original_files\s02.jpg
E:/data_for_seg/z_projection\dead\filled_cleaned\original_files\s03_no_stack.jpg
E:/data_for_seg/z_projection\dead\filled_cleaned\original_files\s03.jpg
E:/data_for_seg/z_projection\dead\filled_cleaned\original_files\s04_no_stack.jpg
E:/data_for_seg/z_projection\dead\f